#### Batch Normalization

In [33]:
import keras.losses
from keras.layers import Flatten, Dense, BatchNormalization
from keras.models import Sequential

In [34]:
model = Sequential([
    Flatten(input_shape=[28, 28]),
    BatchNormalization(),
    Dense(300, activation="relu"),
    BatchNormalization(),
    Dense(100, activation="relu"),
    BatchNormalization(),
    Dense(10, activation="softmax")
])

In [35]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_10 (Bat  (None, 784)              3136      
 chNormalization)                                                
                                                                 
 dense_9 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_11 (Bat  (None, 300)              1200      
 chNormalization)                                                
                                                                 
 dense_10 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_12 (Bat  (None, 100)             

In [36]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization_10/gamma:0', True),
 ('batch_normalization_10/beta:0', True),
 ('batch_normalization_10/moving_mean:0', False),
 ('batch_normalization_10/moving_variance:0', False)]

#### Gradient Clipping

In [38]:
from keras.optimizers import gradient_descent_v2

optimizer = gradient_descent_v2.SGD(clipvalue=1.0)

In [39]:
import numpy as np
from keras.datasets import fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6)  # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2  # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32)  # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))


(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [40]:
X_train_A.shape

(43986, 28, 28)

In [41]:
X_train_B.shape

(200, 28, 28)

In [43]:
model_A = Sequential()
model_A.add(Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(Dense(n_hidden, activation="selu"))
model_A.add(Dense(8, activation="softmax"))

In [44]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [45]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
 149/1375 [==>...........................] - ETA: 1s - loss: 1.1940 - accuracy: 0.6195

2022-03-15 16:25:29.244291: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1375/1375 [==============================] - 2s 980us/step - loss: 0.5579 - accuracy: 0.8250 - val_loss: 0.3726 - val_accuracy: 0.8762
Epoch 2/20
1375/1375 [==============================] - 1s 916us/step - loss: 0.3475 - accuracy: 0.8810 - val_loss: 0.3236 - val_accuracy: 0.8914
Epoch 3/20
1375/1375 [==============================] - 1s 979us/step - loss: 0.3137 - accuracy: 0.8916 - val_loss: 0.2979 - val_accuracy: 0.9016
Epoch 4/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.2952 - accuracy: 0.8989 - val_loss: 0.2844 - val_accuracy: 0.9081
Epoch 5/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.2821 - accuracy: 0.9032 - val_loss: 0.2773 - val_accuracy: 0.9041
Epoch 6/20
1375/1375 [==============================] - 1s 966us/step - loss: 0.2728 - accuracy: 0.9066 - val_loss: 0.2749 - val_accuracy: 0.9046
Epoch 7/20
1375/1375 [==============================] - 1s 951us/step - loss: 0.2649 - accuracy: 0.9095 - val_loss: 0.2718 - val_accuracy: 

In [46]:
model_A.save("my_model_A.h5")

In [47]:
model_B = Sequential()
model_B.add(Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(Dense(n_hidden, activation="selu"))
model_B.add(Dense(1, activation="sigmoid"))

In [48]:
model_B.compile(loss="binary_crossentropy",
                optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [49]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 17ms/step - loss: 0.6125 - accuracy: 0.7000 - val_loss: 0.5306 - val_accuracy: 0.7748
Epoch 2/20
7/7 [==============================] - 0s 10ms/step - loss: 0.4405 - accuracy: 0.8300 - val_loss: 0.4127 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 18ms/step - loss: 0.3394 - accuracy: 0.8850 - val_loss: 0.3372 - val_accuracy: 0.8986
Epoch 4/20
7/7 [==============================] - 0s 10ms/step - loss: 0.2715 - accuracy: 0.9400 - val_loss: 0.2873 - val_accuracy: 0.9209
Epoch 5/20
7/7 [==============================] - 0s 8ms/step - loss: 0.2275 - accuracy: 0.9700 - val_loss: 0.2517 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 8ms/step - loss: 0.1950 - accuracy: 0.9700 - val_loss: 0.2243 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 8ms/step - loss: 0.1701 - accuracy: 0.9850 - val_loss: 0.2014 - val_accuracy: 0.9584
Epoch 8/20
7/7 [==============

In [50]:
model_B.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_18 (Dense)            (None, 300)               235500    
                                                                 
 dense_19 (Dense)            (None, 100)               30100     
                                                                 
 dense_20 (Dense)            (None, 50)                5050      
                                                                 
 dense_21 (Dense)            (None, 50)                2550      
                                                                 
 dense_22 (Dense)            (None, 50)                2550      
                                                                 
 dense_23 (Dense)            (None, 1)                

In [54]:
from keras.models import load_model

model_A: Sequential = load_model("my_model_A.h5")
model_B_on_A = Sequential(model_A.layers[:-1])  # no output layer
model_B_on_A.add(Dense(1, activation="sigmoid"))

# Note that model_B_on_A and model_A share layers now, so when we train one, it will update both models. If we want to avoid that, we need to build model_B_on_A on top of a clone of model_A:

In [55]:
from keras.models import clone_model

model_A_clone = clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(Dense(1, activation="sigmoid"))

In [62]:
for layer in model_B_on_A.layers[:-1]:  # Freezing all hidden layers
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [63]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 19ms/step - loss: 0.0458 - accuracy: 0.9950 - val_loss: 0.0561 - val_accuracy: 0.9939
Epoch 2/4
7/7 [==============================] - 0s 9ms/step - loss: 0.0434 - accuracy: 0.9950 - val_loss: 0.0538 - val_accuracy: 0.9939
Epoch 3/4
7/7 [==============================] - 0s 8ms/step - loss: 0.0414 - accuracy: 0.9950 - val_loss: 0.0518 - val_accuracy: 0.9939
Epoch 4/4
7/7 [==============================] - 0s 8ms/step - loss: 0.0395 - accuracy: 0.9950 - val_loss: 0.0499 - val_accuracy: 0.9939


In [64]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 0s 16ms/step - loss: 0.0381 - accuracy: 0.9950 - val_loss: 0.0489 - val_accuracy: 0.9939
Epoch 2/16
7/7 [==============================] - 0s 17ms/step - loss: 0.0370 - accuracy: 0.9950 - val_loss: 0.0480 - val_accuracy: 0.9939
Epoch 3/16
7/7 [==============================] - 0s 8ms/step - loss: 0.0359 - accuracy: 0.9950 - val_loss: 0.0471 - val_accuracy: 0.9939
Epoch 4/16
7/7 [==============================] - 0s 8ms/step - loss: 0.0350 - accuracy: 0.9950 - val_loss: 0.0463 - val_accuracy: 0.9949
Epoch 5/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0338 - accuracy: 0.9950 - val_loss: 0.0455 - val_accuracy: 0.9949
Epoch 6/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0330 - accuracy: 0.9950 - val_loss: 0.0448 - val_accuracy: 0.9949
Epoch 7/16
7/7 [==============================] - 0s 17ms/step - loss: 0.0322 - accuracy: 0.9950 - val_loss: 0.0441 - val_accuracy: 0.9949
Epoch 8/16
7/7 [===============

In [65]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0971 - accuracy: 0.9815


[0.09714802354574203, 0.9815000295639038]

In [66]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0297 - accuracy: 0.9990


[0.02974965237081051, 0.9990000128746033]

#### Momentum Optimization

In [70]:
_ = gradient_descent_v2.SGD(momentum=0.9)

#### Nesterov Accelerated Gradient

In [71]:
_ = gradient_descent_v2.SGD(momentum=0.9, nesterov=True)

#### AdaGrad

In [73]:
from keras.optimizers import adagrad_v2

_ = adagrad_v2.Adagrad()  # not recommended for nn, since it can stop to early

#### RMSProp

In [77]:
from keras.optimizers import rmsprop_v2

_ = rmsprop_v2.RMSprop(learning_rate=0.001, rho=0.9)

#### Adam and Nadam
nadam is Adam + Nesterov Trick

In [ ]:
from keras.optimizers import adam_v2

_ = adam_v2.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)